### DOWNLOAD DATA

In [ ]:
#!pip install yfinance
import yfinance as yf
import pandas as pd

# Download S&P 500 index data (^GSPC) from 1990-01-01 to 2018-12-31
sp500 = yf.download("^GSPC", start="1990-01-01", end="2018-12-31", interval="1d")

# Save to CSV
sp500.to_csv("sp500_1990_2018.csv")

print("Saved S&P 500 data to sp500_1990_2018.csv")

# Save to CSV
sp500.to_csv("data/sp500_1990_2018.csv")

print("Saved S&P 500 data to sp500_1990_2018.csv")

# download and save VIX
vix = yf.download("^VIX", start="1990-01-01", end="2018-12-31",
                 interval="1d", auto_adjust=False, progress=False)

# Get the "Close" column for ticker ^VIX
vix_close = vix[("Close", "^VIX")].rename("VIX_Close")
vix_df = vix_close.to_frame()

vix_df.to_csv("data/vix_1990_2018.csv")

print("Saved VIX data to vix_1990_2018.csv")

### IMPORT INTO PANDAS FROM DISK

In [1]:
import pandas as pd
import numpy as np

# 1) Load exactly as-is
df = pd.read_csv("data/sp500_1990_2018.csv")

# 2) Drop the two metadata rows: where 'Price' is 'Ticker' or 'Date'
df = df[~df["Price"].isin(["Ticker", "Date"])].copy()

# 3) Parse dates and set index
df["Date"] = pd.to_datetime(df["Price"])
df = df.drop(columns=["Price"]).set_index("Date").sort_index()

# 4) Make sure numeric cols are numeric
for c in ["Close","High","Low","Open","Volume"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 5) Drop intra-day prices
df = df.drop(columns=["High", "Low", "Open"])

print(df.head())
print(df.dtypes)

                 Close     Volume
Date                             
1990-01-02  359.690002  162070000
1990-01-03  358.760010  192330000
1990-01-04  355.670013  177000000
1990-01-05  352.200012  158530000
1990-01-08  353.790009  140110000
Close     float64
Volume      int64
dtype: object


### CLEAN SPX DATA

In [33]:
import pandas as pd

# Load raw CSV
spx_raw = pd.read_csv("data/sp500_1990_2018.csv")

# Drop the two junk rows ("Ticker", "Date")
spx_clean = spx_raw[~spx_raw["Price"].isin(["Ticker","Date"])].copy()

# Parse Date
spx_clean["Date"] = pd.to_datetime(spx_clean["Price"], errors="raise")
spx_clean = spx_clean.drop(columns=["Price"]).set_index("Date").sort_index()

# Ensure numeric types
for c in ["Close","High","Low","Open","Volume"]:
    if c in spx_clean.columns:
        spx_clean[c] = pd.to_numeric(spx_clean[c], errors="coerce")

#print(spx_clean.dtypes)
#print(spx_clean.head())

# Optional: sanity check for technical indicators
need_cols = ["Close","High","Low"]
assert all(c in spx_clean.columns for c in need_cols), "Missing OHLC columns."

spx_clean.head()

,Close,High,Low,Open,Volume
Date,,,,,
1990-01-02,359.690002,359.690002,351.980011,353.399994,162070000
1990-01-03,358.760010,360.589996,357.890015,359.690002,192330000
1990-01-04,355.670013,358.760010,352.890015,358.760010,177000000
1990-01-05,352.200012,355.670013,351.350006,355.670013,158530000
1990-01-08,353.790009,354.239990,350.540009,352.200012,140110000


### IMPORT & CLEAN VFX

In [37]:
vix = pd.read_csv("data/vix_1990_2018.csv")  # or however you saved it

# Parse Date and set as index
vix["Date"] = pd.to_datetime(vix["Date"])
vix = vix.set_index("Date").sort_index()

print(vix.dtypes)
vix.head()

VIX_Close    float64
dtype: object


,VIX_Close
Date,
1990-01-02,17.240000
1990-01-03,18.190001
1990-01-04,19.219999
1990-01-05,20.110001
1990-01-08,20.260000


### IMPORT & CLEAN MOVE
The move metric began in 2002. Not sure why it's listed in the paper if it doesn't cover the entire training range. Will omit it from the model.

In [40]:
import pandas as pd
import yfinance as yf

START, END = "1990-02-12", "2018-10-05"

# 1) Download
mv = yf.download("^MOVE", start=START, end=END, interval="1d", auto_adjust=False, progress=False)

# 2) Inspect exactly what we got (no renaming yet)
print(type(mv), mv.shape)
print(mv.columns)     # often a MultiIndex: ('Close','^MOVE'), etc.
print(mv.index[:3], "...", mv.index[-3:])
print(mv.head())

# 3) Extract a plain Close series based on what you saw
if isinstance(mv.columns, pd.MultiIndex):
    move_close = mv[("Close", "^MOVE")].rename("MOVE_Close")
else:
    move_close = mv["Close"].rename("MOVE_Close")

# 4) Make it a tidy DataFrame with Date index
move = move_close.to_frame().copy()
move.index = pd.to_datetime(move.index)
move = move.sort_index()

print(move.dtypes)
print(move.head())

<class 'pandas.core.frame.DataFrame'> (3917, 6)
MultiIndex([('Adj Close', '^MOVE'),
            (    'Close', '^MOVE'),
            (     'High', '^MOVE'),
            (      'Low', '^MOVE'),
            (     'Open', '^MOVE'),
            (   'Volume', '^MOVE')],
           names=['Price', 'Ticker'])
DatetimeIndex(['2002-11-12', '2002-11-13', '2002-11-14'], dtype='datetime64[ns]', name='Date', freq=None) ... DatetimeIndex(['2018-10-02', '2018-10-03', '2018-10-04'], dtype='datetime64[ns]', name='Date', freq=None)
Price        Adj Close       Close        High         Low        Open Volume
Ticker           ^MOVE       ^MOVE       ^MOVE       ^MOVE       ^MOVE  ^MOVE
Date                                                                         
2002-11-12  120.209999  120.209999  120.209999  120.209999  120.209999      0
2002-11-13  122.879997  122.879997  122.879997  122.879997  122.879997      0
2002-11-14  130.289993  130.289993  130.289993  130.289993  130.289993      0
2002-11-15  1

### IMPORT & CLEAN ADS

In [44]:
import pandas as pd

ads = pd.read_excel("data/ADS_Index_Most_Current_Vintage.xlsx")

# Rename columns
ads = ads.rename(columns={"Unnamed: 0": "Date"})

# Parse Date column
ads["Date"] = pd.to_datetime(ads["Date"], format="%Y:%m:%d")

# Set index
ads = ads.set_index("Date").sort_index()

# Keep only the research period (1990–2018)
ads = ads.loc["1990-01-01":"2018-12-31"]

print(ads.info())
print(ads.head())
print(ads.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10592 entries, 1990-01-01 to 2018-12-31
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ADS_Index  10592 non-null  float64
 1   RECBARS    10592 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 248.2 KB
None
            ADS_Index  RECBARS
Date                          
1990-01-01  -0.178398        0
1990-01-02  -0.196996        0
1990-01-03  -0.213842        0
1990-01-04  -0.228943        0
1990-01-05  -0.242304        0
            ADS_Index  RECBARS
Date                          
2018-12-27  -0.230603        0
2018-12-28  -0.239467        0
2018-12-29  -0.248664        0
2018-12-30  -0.258198        0
2018-12-31  -0.269100        0


### IMPORT & CLEAN FRED RATES

In [45]:
codes = ["DFF","DTB3","DGS10","DGS30","DAAA","DBAA","TEDRATE"]
rates = pdr.DataReader(codes, "fred", START, END)
peek(rates, "Rates/Yields/TED (raw)")

# Rename for clarity
rates = rates.rename(columns={
    "DFF":"FedFunds", "DTB3":"TBill3M", "DGS10":"Govt10Y", "DGS30":"Govt30Y",
    "DAAA":"Aaa", "DBAA":"Baa", "TEDRATE":"TED"
})

# Spreads & daily deltas exactly like appendix
rates_feat = pd.DataFrame(index=rates.index)
rates_feat["fft"]   = rates["FedFunds"].diff()       # Fed funds change
rates_feat["3mth"]  = rates["TBill3M"].diff()
rates_feat["10yr"]  = rates["Govt10Y"].diff()
rates_feat["30yr"]  = rates["Govt30Y"].diff()
rates_feat["Aaa"]   = rates["Aaa"].diff()
rates_feat["Baa"]   = rates["Baa"].diff()
rates_feat["term_spread"]  = rates["Govt10Y"] - rates["TBill3M"]
rates_feat["long_spread"]  = rates["Govt30Y"] - rates["Govt10Y"]
rates_feat["corp_spread"]  = rates["Baa"] - rates["Aaa"]
rates_feat["Aaa_minus_10Y"]= rates["Aaa"] - rates["Govt10Y"]
rates_feat["TED"]          = rates["TED"]  # paper lists TED separately
peek(rates_feat, "Rates features (clean-ish)")


=== Rates/Yields/TED (raw) ===
shape: (10463, 7)
index: <class 'pandas.core.indexes.datetimes.DatetimeIndex'> DatetimeIndex(['1990-02-12', '1990-02-13', '1990-02-14'], dtype='datetime64[ns]', name='DATE', freq='D') … DatetimeIndex(['2018-10-03', '2018-10-04', '2018-10-05'], dtype='datetime64[ns]', name='DATE', freq='D')
columns: <class 'pandas.core.indexes.base.Index'> Index(['DFF', 'DTB3', 'DGS10', 'DGS30', 'DAAA', 'DBAA', 'TEDRATE'], dtype='object')


,DFF,DTB3,DGS10,DGS30,DAAA,DBAA,TEDRATE
DATE,,,,,,,
1990-02-12,8.21,7.73,8.40,8.43,9.20,10.11,0.58
1990-02-13,8.21,7.58,8.35,8.39,9.20,10.10,0.80
1990-02-14,8.21,7.61,8.36,8.41,9.20,10.12,0.70
1990-02-15,8.32,7.71,8.43,8.47,9.21,10.13,0.60
1990-02-16,8.19,7.69,8.42,8.46,9.22,10.10,0.68



=== Rates features (clean-ish) ===
shape: (10463, 11)
index: <class 'pandas.core.indexes.datetimes.DatetimeIndex'> DatetimeIndex(['1990-02-12', '1990-02-13', '1990-02-14'], dtype='datetime64[ns]', name='DATE', freq='D') … DatetimeIndex(['2018-10-03', '2018-10-04', '2018-10-05'], dtype='datetime64[ns]', name='DATE', freq='D')
columns: <class 'pandas.core.indexes.base.Index'> Index(['fft', '3mth', '10yr', '30yr', 'Aaa', 'Baa', 'term_spread',
       'long_spread', 'corp_spread', 'Aaa_minus_10Y', 'TED'],
      dtype='object')


,fft,3mth,10yr,30yr,Aaa,Baa,term_spread,long_spread,corp_spread,Aaa_minus_10Y,TED
DATE,,,,,,,,,,,
1990-02-12,NaN,NaN,NaN,NaN,NaN,NaN,0.67,0.03,0.91,0.80,0.58
1990-02-13,0.00,-0.15,-0.05,-0.04,0.00,-0.01,0.77,0.04,0.90,0.85,0.80
1990-02-14,0.00,0.03,0.01,0.02,0.00,0.02,0.75,0.05,0.92,0.84,0.70
1990-02-15,0.11,0.10,0.07,0.06,0.01,0.01,0.72,0.04,0.92,0.78,0.60
1990-02-16,-0.13,-0.02,-0.01,-0.01,0.01,-0.03,0.73,0.04,0.88,0.80,0.68


### IMPORT & CLEAN Exchange rates (FRED)

In [52]:
from pandas_datareader import data as pdr
import numpy as np
import pandas as pd

START, END = "1990-01-01", "2018-12-31"

# Download major currencies (goods only)
fx_raw = pdr.DataReader("DTWEXM", "fred", START, END)

# Clean into a neat DataFrame
fx = pd.DataFrame(index=fx_raw.index)
fx["majcurr_level"] = fx_raw["DTWEXM"]
fx["majcurr_ret"] = np.log(fx["majcurr_level"]).diff()

# Drop rows with missing values at the start
fx = fx.dropna()

print(fx.info())
print(fx.head())
print(fx.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7139 entries, 1990-01-03 to 2018-12-31
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   majcurr_level  7139 non-null   float64
 1   majcurr_ret    7139 non-null   float64
dtypes: float64(2)
memory usage: 167.3 KB
None
            majcurr_level  majcurr_ret
DATE                                  
1990-01-03        93.2095     0.001500
1990-01-04        91.9952    -0.013113
1990-01-05        91.9587    -0.000397
1990-01-08        91.6824    -0.003009
1990-01-09        92.1002     0.004547
            majcurr_level  majcurr_ret
DATE                                  
2018-12-21        92.1151     0.001841
2018-12-24        92.1151     0.000000
2018-12-27        92.2298     0.002178
2018-12-28        91.9394    -0.003154
2018-12-31        91.7941    -0.001582


### IM[PRT & CLEAN OTHER STOCK MARKET DATA

In [55]:
import pandas as pd
import numpy as np
import yfinance as yf

START, END = "1990-01-01", "2018-12-31"

# Be explicit about auto_adjust to silence the warning
dax_raw  = yf.download("^GDAXI", start=START, end=END, interval="1d", auto_adjust=True, progress=False)
ftse_raw = yf.download("^FTSE",  start=START, end=END, interval="1d", auto_adjust=True, progress=False)

def close_series(df, ticker_label, out_name):
    # Works for both MultiIndex and flat columns
    if isinstance(df.columns, pd.MultiIndex):
        s = df.xs("Close", level=0, axis=1)
        # If still DataFrame (e.g., one column), squeeze to Series
        if isinstance(s, pd.DataFrame):
            # prefer exact ticker match; otherwise first column
            if ticker_label in s.columns:
                s = s[ticker_label]
            else:
                s = s.iloc[:, 0]
    else:
        s = df["Close"]
    s = s.rename(out_name)
    return s.to_frame()

dax  = close_series(dax_raw,  "^GDAXI", "DAX")
ftse = close_series(ftse_raw, "^FTSE",  "FTSE")

# Ensure tz-naive + sorted
for df in (dax, ftse):
    if getattr(df.index, "tz", None) is not None:
        df.index = df.index.tz_localize(None)
    df.sort_index(inplace=True)

# Log returns
rdax  = np.log(dax["DAX"]).diff().rename("rdax").to_frame().dropna()
rftse = np.log(ftse["FTSE"]).diff().rename("rftse").to_frame().dropna()

# Sanity checks
print("DAX level:",  dax.index.min().date(),  "→", dax.index.max().date(),  "| rows:", len(dax))
print("FTSE level:", ftse.index.min().date(), "→", ftse.index.max().date(), "| rows:", len(ftse))
print("rdax:",  rdax.shape,  " NaNs:", rdax.isna().sum().sum())
print("rftse:", rftse.shape, " NaNs:", rftse.isna().sum().sum())
print(dax.head(), "\n", ftse.head())
print(rdax.head(), "\n", rftse.head())

DAX level: 1990-01-02 → 2018-12-28 | rows: 7328
FTSE level: 1990-01-02 → 2018-12-28 | rows: 7325
rdax: (7327, 1)  NaNs: 0
rftse: (7324, 1)  NaNs: 0
                    DAX
Date                   
1990-01-02  1788.890015
1990-01-03  1867.290039
1990-01-04  1830.920044
1990-01-05  1812.900024
1990-01-08  1841.469971 
                    FTSE
Date                   
1990-01-02  2434.100098
1990-01-03  2463.699951
1990-01-04  2451.600098
1990-01-05  2444.500000
1990-01-08  2431.300049
                rdax
Date                
1990-01-03  0.042893
1990-01-04 -0.019670
1990-01-05 -0.009891
1990-01-08  0.015636
1990-01-09  0.012976 
                rftse
Date                
1990-01-03  0.012087
1990-01-04 -0.004923
1990-01-05 -0.002900
1990-01-08 -0.005414
1990-01-09  0.002054


### IMPORT & CLEAN COMMODITIES

In [64]:
gold = pd.read_csv("data/gold_daily_lbma_1990_2018.csv")

# Parse date and set as index
gold["Date"] = pd.to_datetime(gold["date"])
gold = gold.set_index("Date").sort_index()

# Keep only gold_usd
gold = gold[["gold_usd"]].rename(columns={"gold_usd": "Gold"})

print(gold.info())
print(gold.head())
print(gold.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7273 entries, 1990-01-02 to 2018-12-28
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gold    7273 non-null   float64
dtypes: float64(1)
memory usage: 113.6 KB
None
             Gold
Date             
1990-01-02  399.0
1990-01-03  395.0
1990-01-04  396.5
1990-01-05  405.0
1990-01-08  404.6
               Gold
Date               
2018-12-19  1255.00
2018-12-20  1259.75
2018-12-21  1258.15
2018-12-27  1268.00
2018-12-28  1279.00


In [67]:
silver = pd.read_csv("data/silver_daily_lbma_1990_2018.csv")

# Parse date and set as index
silver["Date"] = pd.to_datetime(silver["date"])
silver = silver.set_index("Date").sort_index()

# Keep only gold_usd
silver = silver[["silver_usd"]].rename(columns={"silver_usd": "Silver"})

print(gold.info())
print(gold.head())
print(gold.tail())
silver.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7273 entries, 1990-01-02 to 2018-12-28
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gold    7273 non-null   float64
dtypes: float64(1)
memory usage: 113.6 KB
None
             Gold
Date             
1990-01-02  399.0
1990-01-03  395.0
1990-01-04  396.5
1990-01-05  405.0
1990-01-08  404.6
               Gold
Date               
2018-12-19  1255.00
2018-12-20  1259.75
2018-12-21  1258.15
2018-12-27  1268.00
2018-12-28  1279.00


,Silver
Date,
1990-01-02,5.210
1990-01-03,5.160
1990-01-04,5.196
1990-01-05,5.280
1990-01-08,5.280


In [98]:
import pandas as pd
import numpy as np
import yfinance as yf  # keep if you use it elsewhere

START, END = "1990-01-01", "2018-12-31"

# ---------- WTI (CSV: observation_date, DCOILWTICO) ----------
wti_raw = pd.read_csv("data/DCOILWTICO.csv")
print("WTI preview:\n", wti_raw.head(), "\n")

wti = (
    wti_raw.rename(columns={"observation_date": "Date", "DCOILWTICO": "WTI"})
           .assign(Date=lambda df: pd.to_datetime(df["Date"]))
           .set_index("Date")["WTI"]
           .replace(".", np.nan)
           .astype(float)
           .sort_index()
      ).loc[START:END]

oil = np.log(wti).diff().rename("oil")
print("WTI range:", (wti.index.min(), wti.index.max()), "| rows:", wti.shape[0])
print(oil.dropna().head(), "\n")

# ---------- Copper (monthly, USD/metric ton, CSV: observation_date, PCOPPUSDM) ----------
# --- Copper (monthly → daily on SPX trading days) ---
copper_m = (
    pd.read_csv("data/PCOPPUSDM.csv")
      .rename(columns={"observation_date": "Date", "PCOPPUSDM": "Copper"})
      .assign(Date=lambda df: pd.to_datetime(df["Date"]))
      .set_index("Date")
      .loc["1990-01-01":"2018-12-31"]
)

# 1) Union SPX + monthly dates so 1990-01-01 exists during the fill
# 2) Sort, forward-fill, then slice back to SPX trading days
copper_daily = (
    copper_m.reindex(spx_clean.index.union(copper_m.index))
             .sort_index()
             .ffill()
             .loc[spx_clean.index]
)

# Daily log-returns on the SPX calendar
copper_daily["copper_ret"] = np.log(copper_daily["Copper"]).diff()

print(copper_daily.info())
print(copper_daily.head())

# ---------- Final: join on SPX calendar ----------
# Align OIL to SPX trading days
oil_df = oil.reindex(spx_clean.index).to_frame('oil')

# Use the DAILY copper you created (forward-filled), not the old monthly frame
# Recompute copper_ret AFTER alignment so it’s daily too
copper_daily_aligned = copper_daily.reindex(spx_clean.index)
copper_daily_aligned['copper_ret'] = np.log(copper_daily_aligned['Copper']).diff()

# Final commodity block on SPX calendar
commod = oil_df.join(copper_daily_aligned[['Copper', 'copper_ret']], how='left')

# Quick sanity checks
print(commod[['oil','Copper','copper_ret']].info())
print(commod.head())
print(commod[['Copper','copper_ret']].iloc[:15])

WTI preview:
   observation_date  DCOILWTICO
0       1986-01-02       25.56
1       1986-01-03       26.00
2       1986-01-06       26.53
3       1986-01-07       25.85
4       1986-01-08       25.87 

WTI range: (Timestamp('1990-01-01 00:00:00'), Timestamp('2018-12-31 00:00:00')) | rows: 7566
Date
1990-01-03    0.039842
1990-01-04   -0.016942
1990-01-05   -0.014630
1990-01-08   -0.063990
1990-01-09    0.027799
Name: oil, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7306 entries, 1990-01-02 to 2018-12-28
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Copper      7306 non-null   float64
 1   copper_ret  7305 non-null   float64
dtypes: float64(2)
memory usage: 171.2 KB
None
                 Copper  copper_ret
Date                               
1990-01-02  2365.556991         NaN
1990-01-03  2365.556991         0.0
1990-01-04  2365.556991         0.0
1990-01-05  2365.556991         0.0
1990-0

### INDICATORS

In [85]:
import numpy as np
import pandas as pd

# Work from the cleaned S&P500 data
px = spx_clean.copy()

# Sanity: ensure datetime index
if not isinstance(px.index, pd.DatetimeIndex):
    px.index = pd.to_datetime(px.index)

# --- Parameters ---
n = 10

# --- Returns ---
px["ret"] = np.log(px["Close"]).diff()

# --- Moving Average (MA_n) ---
px["MA10"] = px["Close"].rolling(n).mean()

# --- Momentum (P_t - P_{t-(n-1)}) ---
px["MOM10"] = px["Close"] - px["Close"].shift(n-1)

# --- Stochastic %K and %D ---
Hn = px["High"].rolling(n).max()
Ln = px["Low"].rolling(n).min()
rng = (Hn - Ln).replace(0, np.nan)
px["StochK"] = (px["Close"] - Ln) / rng * 100
px["StochD"] = px["StochK"].rolling(n).mean()

# --- RSI (n=10) ---
delta = px["Close"].diff()
up   = delta.clip(lower=0).rolling(n-1).sum()
down = (-delta.clip(upper=0)).rolling(n-1).sum()
px["RSI"] = 100 * (up / (up + down))

# --- Williams %R ---
px["Rperc"] = (Hn - px["Close"]) / rng * 100

# --- MACD (12,26) and signal (9) ---
ema12 = px["Close"].ewm(span=12, adjust=False).mean()
ema26 = px["Close"].ewm(span=26, adjust=False).mean()
px["MACD"]    = ema12 - ema26
px["MACDsig"] = px["MACD"].ewm(span=9, adjust=False).mean()

# --- Final feature frame ---
tech = px[["ret","MA10","MOM10","StochK","StochD","RSI","Rperc","MACD","MACDsig"]]

print(tech.info())
print(tech.dropna().head())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7306 entries, 1990-01-02 to 2018-12-28
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ret      7305 non-null   float64
 1   MA10     7297 non-null   float64
 2   MOM10    7297 non-null   float64
 3   StochK   7297 non-null   float64
 4   StochD   7288 non-null   float64
 5   RSI      7297 non-null   float64
 6   Rperc    7297 non-null   float64
 7   MACD     7306 non-null   float64
 8   MACDsig  7306 non-null   float64
dtypes: float64(9)
memory usage: 570.8 KB
None
                 ret        MA10      MOM10     StochK     StochD        RSI  \
Date                                                                           
1990-01-26 -0.000859  333.661996 -11.200012  21.195838  16.068316  27.291110   
1990-01-29 -0.001843  332.481998 -15.549988  18.279088  17.717210  13.853964   
1990-01-30 -0.006850  330.704999 -14.419983  14.202079  16.426174  14.622123   
1990-01-31 

### MERGE INTO ONE DF

In [109]:
dfs = [vix, spx_clean, ads, rates_feat, fx, dax, rdax, ftse,
       gold, silver, commod, tech]

merged = pd.concat(dfs, axis=1, join="outer")

# --- Compute gold/silver returns inside merged, then drop level columns ---

# Calculate log-returns from the raw Gold and Silver price columns
if "Gold" in merged.columns:
    merged["gold_ret"] = np.log(merged["Gold"]).diff()

if "Silver" in merged.columns:
    merged["silver_ret"] = np.log(merged["Silver"]).diff()

# Drop the unused level/price columns
cols_to_drop = ["majcurr_level", "Copper", "Gold", "Silver"]
merged = merged.drop(columns=cols_to_drop, errors="ignore")

# Sanity check
print("Columns now include:", [c for c in merged.columns if "gold" in c or "silver" in c])
print(merged[["gold_ret", "silver_ret"]].head())
print(merged.info())

# Drop unnecessary level columns, keep only return-based features
cols_to_drop = ["majcurr_level", "Copper", "Gold", "Silver"]
merged = merged.drop(columns=cols_to_drop, errors="ignore")

print(merged.info())
print(merged.head())

Columns now include: ['gold_ret', 'silver_ret']
            gold_ret  silver_ret
1990-01-01       NaN         NaN
1990-01-02       NaN         NaN
1990-01-03 -0.010076   -0.009643
1990-01-04  0.003790    0.006953
1990-01-05  0.021211    0.016037
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10592 entries, 1990-01-01 to 2018-12-31
Freq: D
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   VIX_Close      7306 non-null   float64
 1   Close          7306 non-null   float64
 2   High           7306 non-null   float64
 3   Low            7306 non-null   float64
 4   Open           7306 non-null   float64
 5   Volume         7306 non-null   float64
 6   ADS_Index      10592 non-null  float64
 7   RECBARS        10592 non-null  int64  
 8   fft            10462 non-null  float64
 9   3mth           5596 non-null   float64
 10  10yr           5596 non-null   float64
 11  30yr           5596 non-null   float64
 12 

### CLEAN
- drops non-trading days
- forward fill missing vals

In [110]:
# ---------- Align & fill strategy (SPX calendar + warm-up + internal gaps only) ----------
# 1) Align everything to SPX trading days
X = merged.reindex(spx_clean.index)

# 2) Warm-up: only fix the first N rows' *leading* NaNs by backfilling within that window
N = 30  # adjust if you want a different warm-up size
X = X.copy()
X.iloc[:N] = X.iloc[:N].bfill()  # carry earliest available values backward *within* the first N rows

# 3) Fill only *internal* gaps (i.e., between two valid values). 
#    This leaves any remaining leading/trailing NaNs intact.
X = X.ffill(limit_area="inside")

# ---------- Quick sanity checks ----------
print("Rows/cols:", X.shape)

# NaNs in the first 10 rows (after warm-up)
print("Start window missing per col (first 10 rows):")
print(X.iloc[:10].isna().sum())

# Any NaNs remaining *after* the warm-up window?
has_nans_afterN = X.iloc[N:].isna().any().any()
print("\nAny NaNs outside the warm-up window?:", has_nans_afterN)

# If any remain, show counts to see which columns still have gaps
if has_nans_afterN:
    na_counts_afterN = X.iloc[N:].isna().sum()
    print("\nNaN counts after warm-up (by column):")
    print(na_counts_afterN[na_counts_afterN > 0].sort_values(ascending=False))

# Peek a few rows
print("\nHead:")
print(X.head(3))
print("\nAround warm-up boundary:")
print(X.iloc[max(0, N-2):N+2])

Rows/cols: (7306, 36)
Start window missing per col (first 10 rows):
VIX_Close         0
Close             0
High              0
Low               0
Open              0
Volume            0
ADS_Index         0
RECBARS           0
fft              10
3mth             10
10yr             10
30yr             10
Aaa              10
Baa              10
term_spread       0
long_spread       0
corp_spread       0
Aaa_minus_10Y     0
TED               0
majcurr_ret       0
DAX               0
rdax              0
FTSE              0
oil               0
copper_ret        0
ret               0
MA10              0
MOM10             0
StochK            0
StochD            0
RSI               0
Rperc             0
MACD              0
MACDsig           0
gold_ret          0
silver_ret        0
dtype: int64

Any NaNs outside the warm-up window?: True

NaN counts after warm-up (by column):
fft              57
3mth             57
10yr             57
30yr             57
Aaa              57
Baa             

In [111]:
# 3) Fill only *internal* gaps using BOTH directions (no leading/trailing touch)
#    This handles isolated holes where a forward fill alone can’t reach.
X = X.bfill(limit_area="inside").ffill(limit_area="inside")

# ---------- Sanity checks (unchanged) ----------
print("Rows/cols:", X.shape)
print("Start window missing per col (first 10 rows):")
print(X.iloc[:10].isna().sum())

has_nans_afterN = X.iloc[N:].isna().any().any()
print("\nAny NaNs outside the warm-up window?:", has_nans_afterN)

if has_nans_afterN:
    na_counts_afterN = X.iloc[N:].isna().sum()
    print("\nNaN counts after warm-up (by column):")
    print(na_counts_afterN[na_counts_afterN > 0].sort_values(ascending=False))

Rows/cols: (7306, 36)
Start window missing per col (first 10 rows):
VIX_Close         0
Close             0
High              0
Low               0
Open              0
Volume            0
ADS_Index         0
RECBARS           0
fft              10
3mth             10
10yr             10
30yr             10
Aaa              10
Baa              10
term_spread       0
long_spread       0
corp_spread       0
Aaa_minus_10Y     0
TED               0
majcurr_ret       0
DAX               0
rdax              0
FTSE              0
oil               0
copper_ret        0
ret               0
MA10              0
MOM10             0
StochK            0
StochD            0
RSI               0
Rperc             0
MACD              0
MACDsig           0
gold_ret          0
silver_ret        0
dtype: int64

Any NaNs outside the warm-up window?: True

NaN counts after warm-up (by column):
fft              57
3mth             57
10yr             57
30yr             57
Aaa              57
Baa             

In [112]:
# --- Targeted fill for columns still NaN after warm-up ---
N = 30  # same as before
afterN = X.iloc[N:]

# which columns still have NaNs after the warm-up window?
cols_still_nan = afterN.columns[afterN.isna().any()].tolist()
print("Columns with NaNs after warm-up:", cols_still_nan)

# backfill leading gaps up to the first valid, and ffill any trailing tail
for c in cols_still_nan:
    s = X[c].copy()

    # 1) fill leading block up to first valid obs
    fv = s.first_valid_index()
    if fv is not None:
        s.loc[:fv] = s.loc[:fv].bfill()

    # 2) fill internal gaps (if any remain)
    s = s.bfill(limit_area="inside").ffill(limit_area="inside")

    # 3) fill any trailing block from last valid obs to end
    lv = s.last_valid_index()
    if lv is not None:
        s.loc[lv:] = s.loc[lv:].ffill()

    X[c] = s

# sanity checks
print("\nAny NaNs outside the warm-up window now?:", X.iloc[N:].isna().any().any())
left = X.iloc[N:].isna().sum()
left = left[left > 0].sort_values(ascending=False)
if not left.empty:
    print("\nRemaining NaNs after warm-up (by column):")
    print(left)
else:
    print("\nAll clear after warm-up window.")

Columns with NaNs after warm-up: ['fft', '3mth', '10yr', '30yr', 'Aaa', 'Baa', 'term_spread', 'long_spread', 'corp_spread', 'Aaa_minus_10Y', 'TED']

Any NaNs outside the warm-up window now?: False

All clear after warm-up window.


In [113]:
# quick double-check
print(X.isna().sum().sum())  # should be 0

# save
X.to_csv("data/merged_features_clean.csv", index=True)

0


### CALCULATE THRESHOLDS

In [119]:
import numpy as np
import pandas as pd

# --- inputs: X is your merged feature DF on SPX trading days and includes 'ret' ---
SPLIT_DATE = pd.Timestamp("2007-01-01")

df = X.copy()
df = df.drop(columns=["Close", "High", "Low", "Open", "Volume"])

# Target (same-day log return); features exclude 'ret'
y = df["ret"]
F = df.drop(columns=["ret"])

# Train/test masks (no shuffling)
train_mask = df.index < SPLIT_DATE
test_mask  = ~train_mask

# --- Quantiles from TRAIN ONLY ---
q1, q3 = y[train_mask].quantile([0.25, 0.75])
print(f"Train Q1={q1:.6f}, Q3={q3:.6f}")

# --- Fixed-threshold labeling for ALL DATES ---
# label = -1 (<=Q1), 0 (between), +1 (>=Q3); NaN stays NaN if y is NaN
y_lbl = pd.Series(np.select(
    condlist=[y <= q1, y >= q3],
    choicelist=[-1, 1],
    default=0
), index=y.index)
y_lbl[y.isna()] = np.nan  # ensure missing returns remain missing

# --- Split ---
X_train, y_train = F[train_mask], y_lbl[train_mask]
X_test,  y_test  = F[test_mask],  y_lbl[test_mask]

# --- Drop NaNs within each split (features or label) ---
train_keep = X_train.dropna().index.intersection(y_train.dropna().index)
test_keep  = X_test.dropna().index.intersection(y_test.dropna().index)

X_train, y_train = X_train.loc[train_keep], y_train.loc[train_keep]
X_test,  y_test  = X_test.loc[test_keep],  y_test.loc[test_keep]

# --- Quick sanity checks ---
print("\nShapes:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test: ", X_test.shape,  "y_test: ", y_test.shape)

print("\nClass balance (train):")
print(y_train.value_counts().sort_index())

print("\nClass balance (test):")
print(y_test.value_counts().sort_index())

# If you want the thresholds for later use:
Q_THRESHOLDS = {"q1": float(q1), "q3": float(q3)}
print("\nSaved thresholds:", Q_THRESHOLDS)

Train Q1=-0.004650, Q3=0.005508

Shapes:
X_train: (4287, 30) y_train: (4287,)
X_test:  (3019, 30) y_test:  (3019,)

Class balance (train):
-1    1072
 0    2143
 1    1072
Name: count, dtype: int64

Class balance (test):
-1     706
 0    1558
 1     755
Name: count, dtype: int64

Saved thresholds: {'q1': -0.004649549938577913, 'q3': 0.005508441474124659}


In [120]:
print("Row range:", X.index.min(), "→", X.index.max())
print("Rows/cols:", X.shape)

print("\nColumns:")
print(list(X.columns))

print("\nNaN counts per column:")
print(X.isna().sum())

Row range: 1990-01-02 00:00:00 → 2018-12-28 00:00:00
Rows/cols: (7306, 36)

Columns:
['VIX_Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'ADS_Index', 'RECBARS', 'fft', '3mth', '10yr', '30yr', 'Aaa', 'Baa', 'term_spread', 'long_spread', 'corp_spread', 'Aaa_minus_10Y', 'TED', 'majcurr_ret', 'DAX', 'rdax', 'FTSE', 'oil', 'copper_ret', 'ret', 'MA10', 'MOM10', 'StochK', 'StochD', 'RSI', 'Rperc', 'MACD', 'MACDsig', 'gold_ret', 'silver_ret']

NaN counts per column:
VIX_Close        0
Close            0
High             0
Low              0
Open             0
Volume           0
ADS_Index        0
RECBARS          0
fft              0
3mth             0
10yr             0
30yr             0
Aaa              0
Baa              0
term_spread      0
long_spread      0
corp_spread      0
Aaa_minus_10Y    0
TED              0
majcurr_ret      0
DAX              0
rdax             0
FTSE             0
oil              0
copper_ret       0
ret              0
MA10             0
MOM10            0
S

In [121]:
# save to disk
X_train.to_csv("data/X_train.csv", index=True)
y_train.to_csv("data/y_train.csv", index=True, header=True)
X_test.to_csv("data/X_test.csv", index=True)
y_test.to_csv("data/y_test.csv", index=True, header=True)